<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.3:
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and database maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands".

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()
conn.commit()
conn.close()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [2]:
import sqlite3

##Create Table

def create_table():
    # Connect to the SQLite database
    conn = sqlite3.connect(sqlite_db)
    cursor = conn.cursor()

    # Create the table if it doesn't already exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS house (
            soft_area REAL,
            Bdrms INTEGER,
            age INTEGER,
            price REAL,
            house_id NUMERIC PRIMARY KEY
        )
    ''')

    # Commit changes and close the connection
    conn.commit()
    conn.close()

create_table()

### Function to check if Table name exists and get all its column Names

In [3]:
def get_column_names(table_name, db_name):
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_name) #Create Connection
        cursor = conn.cursor() #Create Cusrsor
        
        # Execute PRAGMA table_info to get the column details
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        
        # Check if the table exists by seeing if columns were returned
        if not columns:
            raise ValueError(f"Table '{table_name}' does not exist or has no columns.")
        
        # Extract column names from the results
        column_names = [column[1] for column in columns]
        
        # Close the cursor and connection
        cursor.close()
        conn.close()
        
        return column_names
    
    except sqlite3.Error as e:
        # Handle SQLite errors (e.g., connection errors, query errors)
        print(f"SQLite error: {e}")
        return None
    
    except ValueError as ve:
        # Handle value errors (e.g., table does not exist)
        print(ve)
        return None
    
    except Exception as ex:
        # Handle any other exceptions
        print(f"An error occurred: {ex}")
        return None


# Replace 'your_table_name' with the name of the table you want to describe
get_column_names('house', sqlite_db)

['soft_area', 'Bdrms', 'age', 'price', 'house_id']

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [4]:
# A
last_sale = (4000, 5, 22, 619000, 100)

def insert_values(soft_area, bdrms, age, price, house_id):
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(sqlite_db)
        cursor = conn.cursor()

        # Insert values into the house table
        cursor.execute('''
            INSERT INTO house (soft_area, Bdrms, age, price, house_id)
            VALUES (?, ?, ?, ?, ?)
        ''', (soft_area, bdrms, age, price, house_id))

        # Commit changes and close the connection
        conn.commit()
        conn.close()

        print("Values inserted successfully.")

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    except Exception as ex:
        print(f"An error occurred: {ex}")

insert_values(4000, 5, 22, 619000, 100)

SQLite error: UNIQUE constraint failed: house.house_id


Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key.

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [5]:
# A

recent_sales = [
  (2390, 4, 34, 319000,101),
  (1870, 3, 14, 289000,102),
  (1505, 3, 90, 269000,103),
]

import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()


# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
#
c.executemany('INSERT INTO house VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()
conn.close()

OperationalError: database is locked

Select all rows from houses

In [ ]:
# A
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()
query = 'Select * from house'
c.execute(query)
c.fetchall()
#pd.DataFrame(c.fetchall())

[(4000.0, 5, 22, 619000.0, 100),
 (2390.0, 4, 34, 319000.0, 101),
 (1870.0, 3, 14, 289000.0, 102),
 (1505.0, 3, 90, 269000.0, 103)]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more.

In [ ]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('housing-data.csv', dtype = 'i8',
                    delimiter = ',', skip_header = 1).tolist()

In [ ]:
print(type(data))
print(len(data))

<class 'list'>
47


*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [ ]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[2104, 3, 70, 399900, 108], [1600, 3, 28, 329900, 109], [2400, 3, 44, 369000, 110]]


Now we can insert each list item as a row of fields in the database.

In [ ]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO house VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [ ]:
#import numpy as np
#d1 = np.asarray([1200, 3, 15, 250000])
#d1 = d1.tolist()
#d1.insert(0, None)
#d1
#c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
#conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [ ]:
c.execute('DELETE FROM house WHERE house_id IN (103, 104)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [ ]:
# ANSWER
results = c.execute("SELECT * from house where bdrms = 4")
results.fetchall()

[(2390.0, 4, 34, 319000.0, 101),
 (3000.0, 4, 75, 539900.0, 112),
 (1985.0, 4, 61, 299900.0, 113),
 (1940.0, 4, 7, 239999.0, 118),
 (2300.0, 4, 77, 449900.0, 123),
 (2609.0, 4, 5, 499998.0, 126),
 (3031.0, 4, 21, 599000.0, 127),
 (1962.0, 4, 53, 259900.0, 131),
 (2040.0, 4, 75, 314900.0, 140),
 (1811.0, 4, 24, 285900.0, 142),
 (2132.0, 4, 28, 345000.0, 145),
 (4215.0, 4, 66, 549000.0, 146),
 (2162.0, 4, 43, 287000.0, 147),
 (2567.0, 4, 57, 314000.0, 150),
 (1852.0, 4, 64, 299900.0, 153)]

**2. The following query calculates the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [ ]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(soft_area) AS avg_sqft, AVG(price) AS avg_price FROM house GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1839.9615384615386, 324965.3076923077),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [ ]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database.

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [ ]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('housing-data.csv', low_memory = False)
data.head()

,soft_area,Bdrms,age,price,house_id
0,2104,3,70,399900,108
1,1600,3,28,329900,109
2,2400,3,44,369000,110
3,1416,2,49,232000,111
4,3000,4,75,539900,112


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [ ]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)
conn.commit()

Run a query to get the average price of each house size from this table:

In [ ]:
#ANSWER
results = c.execute("SELECT AVG(price) AS avg_price FROM houses_pandas")
results.fetchall()

[(340412.6595744681,)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate functions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [ ]:
# Reading CSV to Dataframe
orders = pd.read_csv('P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('P12-OrderBreakdown.csv', encoding = 'utf-8')

In [ ]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [ ]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [ ]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [ ]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes

In [ ]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [ ]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [ ]:
# Establishing Local DB connection
db_connection = sqlite3.connect(sqlite_db)

# Save these two dataframes as a table in sqlite

In [ ]:
orders.to_sql('orders', con = conn, if_exists = 'replace', index = False)
conn.commit()
orders_break_down.to_sql('orders_break_down', con = conn, if_exists = 'replace', index = False)
conn.commit()

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [ ]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT *
        FROM orders
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=conn)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [ ]:
orders.head()

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [ ]:
# Select Number of Distinct Orders for Each Customer
results = c.execute("SELECT customer_name, COUNT(DISTINCT order_id) as num_orders FROM orders GROUP BY customer_name")
results.fetchall()

[('Aaron Bergman', 11),
 ('Aaron Hawkins', 7),
 ('Aaron Smayling', 12),
 ('Adam Bellavance', 4),
 ('Adam Hart', 8),
 ('Adam Shillingsburg', 5),
 ('Adrian Barton', 5),
 ('Adrian Hane', 5),
 ('Adrian Shami', 2),
 ('Aimee Bixby', 6),
 ('Alan Barnes', 9),
 ('Alan Dominguez', 5),
 ('Alan Haines', 7),
 ('Alan Hwang', 9),
 ('Alan Schoenberger', 6),
 ('Alan Shonely', 5),
 ('Alejandro Ballentine', 6),
 ('Alejandro Grove', 2),
 ('Alejandro Savely', 7),
 ('Aleksandra Gannaway', 7),
 ('Alex Avila', 5),
 ('Alex Grayson', 8),
 ('Alex Russell', 7),
 ('Alice McCarthy', 7),
 ('Allen Armold', 2),
 ('Allen Goldenen', 2),
 ('Allen Rosenblatt', 6),
 ('Alyssa Crouse', 5),
 ('Alyssa Tate', 3),
 ('Amy Cox', 1),
 ('Amy Hunt', 2),
 ('Andrew Allen', 6),
 ('Andrew Gjertsen', 10),
 ('Andrew Roberts', 5),
 ('Andy Gerbode', 8),
 ('Andy Reiter', 1),
 ('Andy Yotov', 2),
 ('Anemone Ratner', 4),
 ('Angele Hood', 5),
 ('Ann Blume', 5),
 ('Ann Chong', 4),
 ('Ann Steele', 4),
 ('Anna Andreadi', 7),
 ('Anna Chung', 5),
 ('A

#### 6. Select Number of Customers for Each Country

In [ ]:
# Select Number of Customers for Each Country
results = c.execute("SELECT country, COUNT(customer_name) as num_customers FROM orders GROUP BY country")
results.fetchall()

[('Austria', 135),
 ('Belgium', 68),
 ('Denmark', 29),
 ('Finland', 34),
 ('France', 991),
 ('Germany', 806),
 ('Ireland', 50),
 ('Italy', 493),
 ('Netherlands', 194),
 ('Norway', 37),
 ('Portugal', 37),
 ('Spain', 403),
 ('Sweden', 100),
 ('Switzerland', 40),
 ('United Kingdom', 700)]

In [ ]:
orders_break_down.head()

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [ ]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
results = c.execute("SELECT order_id , discount, sales, quantity FROM orders_break_down GROUP BY order_id")
results.fetchall()

[('ES-2011-1001989', 0.1, 229.0, 9),
 ('ES-2011-1010958', 0.0, 58.0, 2),
 ('ES-2011-1012469', 0.0, 148.0, 3),
 ('ES-2011-1043483', 0.0, 224.0, 11),
 ('ES-2011-1058269', 0.0, 13.0, 2),
 ('ES-2011-1084683', 0.0, 63.0, 2),
 ('ES-2011-1088527', 0.0, 76.0, 4),
 ('ES-2011-1119145', 0.0, 18.0, 2),
 ('ES-2011-1122594', 0.0, 58.0, 2),
 ('ES-2011-1136913', 0.0, 84.0, 4),
 ('ES-2011-1137137', 0.1, 599.0, 5),
 ('ES-2011-1138719', 0.0, 22.0, 3),
 ('ES-2011-1140474', 0.0, 15.0, 2),
 ('ES-2011-1141056', 0.1, 593.0, 7),
 ('ES-2011-1196075', 0.0, 50.0, 5),
 ('ES-2011-1205554', 0.0, 82.0, 3),
 ('ES-2011-1210983', 0.0, 351.0, 8),
 ('ES-2011-1222857', 0.6, 393.0, 5),
 ('ES-2011-1228490', 0.1, 69.0, 1),
 ('ES-2011-1244060', 0.0, 728.0, 2),
 ('ES-2011-1259965', 0.0, 152.0, 3),
 ('ES-2011-1273737', 0.0, 31.0, 2),
 ('ES-2011-1276768', 0.1, 438.0, 4),
 ('ES-2011-1298103', 0.0, 590.0, 3),
 ('ES-2011-1322522', 0.0, 64.0, 2),
 ('ES-2011-1386961', 0.0, 15.0, 2),
 ('ES-2011-1401137', 0.0, 114.0, 6),
 ('ES-2011-1416

#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [ ]:
# Select discount, sales, quantity, total price for Each Order Id from orders_break_down Table
results = c.execute("SELECT order_id , (sales*quantity - discount) AS TOTAL_PRICE FROM orders_break_down GROUP BY order_id")
results.fetchall()

[('ES-2011-1001989', 2060.9),
 ('ES-2011-1010958', 116.0),
 ('ES-2011-1012469', 444.0),
 ('ES-2011-1043483', 2464.0),
 ('ES-2011-1058269', 26.0),
 ('ES-2011-1084683', 126.0),
 ('ES-2011-1088527', 304.0),
 ('ES-2011-1119145', 36.0),
 ('ES-2011-1122594', 116.0),
 ('ES-2011-1136913', 336.0),
 ('ES-2011-1137137', 2994.9),
 ('ES-2011-1138719', 66.0),
 ('ES-2011-1140474', 30.0),
 ('ES-2011-1141056', 4150.9),
 ('ES-2011-1196075', 250.0),
 ('ES-2011-1205554', 246.0),
 ('ES-2011-1210983', 2808.0),
 ('ES-2011-1222857', 1964.4),
 ('ES-2011-1228490', 68.9),
 ('ES-2011-1244060', 1456.0),
 ('ES-2011-1259965', 456.0),
 ('ES-2011-1273737', 62.0),
 ('ES-2011-1276768', 1751.9),
 ('ES-2011-1298103', 1770.0),
 ('ES-2011-1322522', 128.0),
 ('ES-2011-1386961', 30.0),
 ('ES-2011-1401137', 684.0),
 ('ES-2011-1416586', 1764.0),
 ('ES-2011-1417751', 352.0),
 ('ES-2011-1418466', 880.0),
 ('ES-2011-1419565', 132.0),
 ('ES-2011-1426891', 711.9),
 ('ES-2011-1436413', 711.9),
 ('ES-2011-1441570', 423.85),
 ('ES-2011

#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [ ]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
results = c.execute("SELECT order_id , (sales*quantity - discount) AS TOTAL_PRICE FROM orders_break_down WHERE TOTAL_PRICE >100 GROUP BY order_id")
results.fetchall()

[('ES-2011-1001989', 2060.9),
 ('ES-2011-1010958', 116.0),
 ('ES-2011-1012469', 444.0),
 ('ES-2011-1043483', 2464.0),
 ('ES-2011-1058269', 2991.9),
 ('ES-2011-1084683', 126.0),
 ('ES-2011-1088527', 304.0),
 ('ES-2011-1122594', 116.0),
 ('ES-2011-1136913', 336.0),
 ('ES-2011-1137137', 2994.9),
 ('ES-2011-1141056', 4150.9),
 ('ES-2011-1196075', 250.0),
 ('ES-2011-1205554', 246.0),
 ('ES-2011-1210983', 2808.0),
 ('ES-2011-1222857', 1964.4),
 ('ES-2011-1228490', 720.0),
 ('ES-2011-1244060', 1456.0),
 ('ES-2011-1259965', 456.0),
 ('ES-2011-1276768', 1751.9),
 ('ES-2011-1298103', 1770.0),
 ('ES-2011-1322522', 128.0),
 ('ES-2011-1401137', 684.0),
 ('ES-2011-1416586', 1764.0),
 ('ES-2011-1417751', 352.0),
 ('ES-2011-1418466', 880.0),
 ('ES-2011-1419565', 132.0),
 ('ES-2011-1426891', 711.9),
 ('ES-2011-1436413', 711.9),
 ('ES-2011-1441570', 423.85),
 ('ES-2011-1443340', 234.0),
 ('ES-2011-1444426', 4088.0),
 ('ES-2011-1455810', 126.9),
 ('ES-2011-1456978', 928.0),
 ('ES-2011-1458159', 5438.9),


#### 8. Select All Customers And The Products They Have Bought

In [ ]:
# Select All Order Ids, Customers And The Products (product_name) They Have Bought
results = c.execute("SELECT od.order_id, od.customer_name, ob.product_name from Orders Od , orders_break_down ob where od.order_id = ob.order_id")
results.fetchall()

[('IT-2011-3647632', 'Eugene Moren', 'Enermax Note Cards, Premium'),
 ('ES-2011-4869686',
  'Dorothy Dickinson',
  'Dania Corner Shelving, Traditional'),
 ('ES-2011-4939443',
  'Arthur Prichep',
  'Binney & Smith Sketch Pad, Easy-Erase'),
 ('IT-2011-2942451', 'Grant Thornton', 'Boston Markers, Easy-Erase'),
 ('IT-2011-2942451', 'Grant Thornton', 'Eldon Folders, Single Width'),
 ('ES-2011-3848439',
  'Michael Granlund',
  'Binney & Smith Pencil Sharpener, Water Color'),
 ('ES-2011-3848439', 'Michael Granlund', 'Sanford Canvas, Fluorescent'),
 ('ES-2011-5433855', 'Dave Poirier', 'Accos Thumb Tacks, Assorted Sizes'),
 ('ES-2011-5433855', 'Dave Poirier', 'Bush Floating Shelf Set, Pine'),
 ('ES-2011-5433855', 'Dave Poirier', 'Smead Lockers, Industrial'),
 ('IT-2011-4546695', 'Darren Powers', 'Binney & Smith Sketch Pad, Blue'),
 ('IT-2011-4546695', 'Darren Powers', 'Ikea Classic Bookcase, Metal'),
 ('ES-2011-1138719', 'Eric Murdock', 'Ibico Hole Reinforcements, Recycled'),
 ('ES-2011-1466305

#### 9.A Select Number of 'Furniture' Orders For Each Country

In [ ]:
# Select Number of 'Furniture' Orders For Each Country
results = c.execute("SELECT COUNT(od.order_id), od.country from Orders Od , orders_break_down ob \
                    where od.order_id = ob.order_id and ob.product_name LIKE '%Ikea%' GROUP BY country")
results.fetchall()

[(2, 'Austria'),
 (2, 'Belgium'),
 (12, 'France'),
 (17, 'Germany'),
 (1, 'Ireland'),
 (14, 'Italy'),
 (6, 'Netherlands'),
 (9, 'Spain'),
 (3, 'Sweden'),
 (9, 'United Kingdom')]

#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [ ]:
results = c.execute("SELECT DISTINCT country from orders")
results.fetchall()

[('Sweden',),
 ('United Kingdom',),
 ('France',),
 ('Italy',),
 ('Austria',),
 ('Spain',),
 ('Germany',),
 ('Netherlands',),
 ('Denmark',),
 ('Belgium',),
 ('Norway',),
 ('Portugal',),
 ('Switzerland',),
 ('Ireland',),
 ('Finland',)]

In [ ]:
# Select Number of 'Furniture' Orders For The Country Denmark
results = c.execute("SELECT COUNT(od.order_id), od.country from Orders Od , orders_break_down ob \
                    where od.order_id = ob.order_id and ob.product_name LIKE '%Ikea%' and \
                    od.country IN ('Denmark') GROUP BY od.country")
results.fetchall()

#There is no country Denmark in the table , hence result is 0

[]

In [ ]:
orders_break_down.head()

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [ ]:
# Select Total Sales With Discount (discount > 0) and Without Discount (discount = 0) for Each Country
results = c.execute("SELECT SUM(ob.sales) AS TOTAL_SALES_WITH_DISOUNT, od.country from Orders Od , orders_break_down ob  \
 where od.order_id = ob.order_id and ob.discount > 0 GROUP BY od.country")
results.fetchall()

[(7763.0, 'Denmark'),
 (479892.0, 'France'),
 (243257.0, 'Germany'),
 (15998.0, 'Ireland'),
 (78645.0, 'Italy'),
 (70313.0, 'Netherlands'),
 (15106.0, 'Portugal'),
 (84507.0, 'Spain'),
 (30490.0, 'Sweden'),
 (89643.0, 'United Kingdom')]

In [ ]:
# Select Total Sales With Discount (discount > 0) and Without Discount (discount = 0) for Each Country
results = c.execute("SELECT SUM(ob.sales) AS TOTAL_SALES_WITH_DISOUNT, od.country from Orders Od , orders_break_down ob  \
 where od.order_id = ob.order_id and ob.discount = 0 GROUP BY od.country")
results.fetchall()

[(79382.0, 'Austria'),
 (42320.0, 'Belgium'),
 (20702.0, 'Finland'),
 (129791.0, 'France'),
 (245424.0, 'Germany'),
 (174097.0, 'Italy'),
 (20529.0, 'Norway'),
 (164895.0, 'Spain'),
 (24874.0, 'Switzerland'),
 (330854.0, 'United Kingdom')]

#### 11.A Select Total Quantity, Total Sales for Each Country

In [ ]:
# Select Total Quantity, Total Sales for Each Country
results = c.execute("SELECT SUM(ob.sales) , SUM(ob.quantity), od.country from Orders Od , orders_break_down ob  \
 where od.order_id = ob.order_id and ob.discount = 0 GROUP BY od.country")
results.fetchall()

[(79382.0, 973, 'Austria'),
 (42320.0, 532, 'Belgium'),
 (20702.0, 201, 'Finland'),
 (129791.0, 4084, 'France'),
 (245424.0, 3958, 'Germany'),
 (174097.0, 2581, 'Italy'),
 (20529.0, 261, 'Norway'),
 (164895.0, 2113, 'Spain'),
 (24874.0, 308, 'Switzerland'),
 (330854.0, 3545, 'United Kingdom')]

#### 11.B Select Top 3 Countries and Quantities Based on Total Sales

In [ ]:
# Select Top 3 Countries and Quantities Based on Total Sales
results = c.execute("SELECT SUM(ob.sales) AS total_sales , SUM(ob.quantity), od.country from Orders Od , orders_break_down ob \
 where od.order_id = ob.order_id  GROUP BY od.country ORDER BY  total_sales DESC LIMIT 3")
results.fetchall()

[(609683.0, 7329, 'France'),
 (488681.0, 6179, 'Germany'),
 (420497.0, 4917, 'United Kingdom')]

#### 11.C Select Bottom 3 Countries and Sales Based On Total Quantities

In [ ]:
# Select Bottom 3 Countries and Sales Based On Total Quantities
results = c.execute("SELECT SUM(ob.sales) AS total_sales , SUM(ob.quantity), od.country from Orders Od , orders_break_down ob \
 where od.order_id = ob.order_id  GROUP BY od.country ORDER BY  total_sales ASC LIMIT 3")
results.fetchall()

[(7763.0, 204, 'Denmark'),
 (15106.0, 286, 'Portugal'),
 (15998.0, 392, 'Ireland')]

#### 12. Select Average Sales By Category For The Country 'France'

In [ ]:
# Select Average Sales By Category For The Country 'France'
results = c.execute("SELECT AVG(ob.sales) AS Avg_sales , ob.category from Orders Od , orders_break_down ob \
 where od.order_id = ob.order_id  and od.country IN ('France') GROUP BY ob.category")
results.fetchall()

[(582.314381270903, 'Furniture'),
 (167.21770917952884, 'Office Supplies'),
 (595.1450777202073, 'Technology')]

#### 13. Select Country, Category and Average Sales Where Average Sales is The Highest

In [ ]:
# Select Country, Category and Average Sales Where Average Sales is The Highest
results = c.execute("SELECT AVG(ob.sales) AS Avg_sales , ob.category , od.country from Orders Od , orders_break_down ob \
 where od.order_id = ob.order_id  GROUP BY ob.category ORDER BY  Avg_sales DESC LIMIT 1")
results.fetchall()

[(581.7564018384767, 'Technology', 'Germany')]

### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams.
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  



---



---



> > > > > > > > > © 2023 Institute of Data


---



---



